# Space 🚀 Missions Data Analysis
This notebook is divided into:

- Data manipulation

- Study about the cost of the missions (fail)

- Study abount the collaboration between companies (fail)

- Launches until first success

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#data type
from datetime import datetime

#for plots
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns


        
        
Space_data = pd.read_csv("../input/all-space-missions-from-1957/Space_Corrected.csv")
Space_data.head()       

# Data manipulation

First I'm going to make a little data treatment.

Location:

I'm going to find the country where the missil is launched from the last part of the column Location. It's not allways true, but we can make an idea of wich countries have their own missiles.

In [ ]:
country=[]
for i in range(0,Space_data.shape[0]):
    str=Space_data['Location'].iloc[i]
    str_by_parts=str.split(",")
    country.append(str_by_parts[len(str_by_parts)-1].strip())

Space_data['Country'] = country

Datum:

From Datum create Date and Year, but later I will only take in to account the Year.

In [ ]:
date=[]
time=[]
year=[]
for i in range(0,Space_data.shape[0]):
    str=Space_data['Datum'].iloc[i]
    day=str[:3]
    mm=str[4:7]
    dd=str[8:10]
    aaaa=str[12:16]
    hh_mm=str[17:22]
    str = dd+'/'+mm+'/'+aaaa
    date_object = datetime.strptime(str, '%d/%b/%Y')
    date.append(date_object)
    year.append(aaaa)


Space_data['Date'] = date
Space_data['Year'] = year

# Convert to numeric (the column was a string)
Space_data['Year']=pd.to_numeric(Space_data.Year)

Status Rocket:

I just convert it into a numeric value.

In [ ]:
status_mapping={'StatusRetired':0,'StatusActive':1}

Space_data['Status Rocket']=Space_data['Status Rocket'].map(status_mapping)

Rocket: cost of the mission in $ million

In [ ]:
# There are spaces after the data 
Space_data['Rocket']=Space_data[' Rocket'].str.strip()

# The thousands mark is a comma
Space_data['Rocket'] = Space_data['Rocket'].str.replace(',','')

# There are nans, so I fill with 0. 
Space_data['Rocket'] = Space_data['Rocket'].fillna(0)

# Convert to numeric (the column was a string)
Space_data['Rocket']=pd.to_numeric(Space_data.Rocket)

# Cost of the missions

Having data of the Space Missions since 1958, I thought it was interesting to study the investment done from the Western Bloc and the Eastern Bloc. 
Althought we don't have the column in the dataset that distinguishes one from another, we can find that information on the internet:

https://en.wikipedia.org/wiki/Western_Bloc

https://en.wikipedia.org/wiki/Eastern_Bloc

The problem that I had to study this is that many of the ussr data don't have this information.

Plot a histogram by countries to make an idea of wich contributes more or less.

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(8,8))
plt.title("Space Missions of each country")
sns.countplot(y="Country", data=Space_data, order=Space_data["Country"].value_counts().index);


We can see how Russia is the country that more launches has done in history. But in fact, during the Cold War, Russia was a part of the USSR as it was Kazakhstan

https://en.wikipedia.org/wiki/Kazakh_Soviet_Socialist_Republic

So, from the point of view of the economic investment during the Cold War, we should include Kazakhstan. In fact, the Sputnik (the first artificial satellite of the USSR and of the world) was launched from Kazakhstan.

I'm going to plot every company (not country) taking into account if we know the investment or not. 

In [ ]:
mask_0=Space_data.Rocket==0

import seaborn as sns
import matplotlib.pyplot as plt

# Initialize the matplotlib figure
sns.set_style(style="whitegrid")
f,ax = plt.subplots(figsize=(15,12))

# Plot the total missions
sns.set_color_codes(palette="bright")
sns.countplot(y="Company Name", data=Space_data,
            label="Total Missions", color="b");
# Plot missions where we don't know the cost
sns.countplot(y="Company Name", data=Space_data[mask_0],
            label="Missions Without Cost", color="r");

# Add a legend and informative axis label
ax.legend(ncol=1, loc="lower right", frameon=True)
ax.set(ylabel="",xlabel="Number of missions")
sns.despine(left=True, bottom=True)
plt.title("Cost of the missions")

We can see how in our dataset we don't have all the investment from the USSR company during all the Cold War.

From the dissolution of the USSR, in 1991, Russia has it's own company, Rocosmos. 

https://en.wikipedia.org/wiki/Roscosmos

In this company we have most of the information about their investment, but there are much less information than during all the cold war. 

If we plot again but after the end of the Cold War (aproximately 1991), we still find an important lack of information about the cost of the missions.

In [ ]:
mask_0=Space_data.Rocket==0
mask_1991=Space_data.Year>=1991


import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style(style="darkgrid")

# Initialize the matplotlib figure
f,ax = plt.subplots(figsize=(12,12))

# Plot the total missions
sns.set_color_codes(palette="muted")
sns.countplot(y="Company Name", data=Space_data,
            label="Total Missions", color="b");
sns.countplot(y="Company Name", data=Space_data[mask_0&mask_1991],
            label="Missions Without Cost", color="r");


# Add a legend and informative axis label
ax.legend(ncol=1, loc="lower right", frameon=True)
ax.set(ylabel="",
       xlabel="Number of missions")
sns.despine(left=True, bottom=True)

# Collaboration between companies

To study the collaborations between different companies in the same missions, I'm going to look up for registers of missions with two or more companies. 

To do that, first I capture the number of times that a Detail appears for different Companies. Then I print the registers that appear more than one time.

In [ ]:
counts=Space_data.groupby('Detail')['Company Name'].transform('nunique')

ind=[]
for i in range(0,counts.shape[0]):
    if counts[i]>1:
        ind.append(i)

Space_data.loc[ind,['Company Name','Detail','Year']].sort_values('Year').sort_values('Detail')


This mission Titan is the only one in all the dataset in wich contributes more than one Company. 

But if we look into the internet, that in 1995 Martin Marietta fussions with Lockheed Corporation to create Lockheed Martin, so this missions don't represent no collaboration. 

https://en.wikipedia.org/wiki/Lockheed_Martin

We can find collaborations between agencies, for example

https://www.esa.int/Science_Exploration/Space_Science/Cassini-Huygens/International_collaboration#:~:text=Cassini-Huygens%20is%20an%20international%20collaboration%20between%20three%20space,Laboratory.%20The%20Huygens%20probe%20was%20built%20by%20ESA.

In fact, if I look in the dataste for Cassini, I find one row


In [ ]:
maskC=Space_data['Detail'].str.contains('Cassini', regex=False)
Space_data[maskC]

Space_data.loc[1360,:]

I think the Dataset contains the information about the launch done by Lockheed. But ESA built the Huygens probe, and ASI provided Cassini's high-gain communication antenna. 

So, there is a collaboration between three companies, but the rocket launch is done just by one of them. That's why it doesn't appear in the dataset as I expected.

We also know that there has been at least one collaboration between USA and USSR, 

https://es.wikipedia.org/wiki/Programa_Apolo

The last Apolo Mission was the Apolo-Soyuz


In [ ]:
Space_data.loc[2911,:]

In [ ]:
Space_data.loc[2912,:]

I was searching for collaborations using the Detail to find coincidences, but it seems better to use the Date.

In [ ]:
counts=Space_data.groupby('Date')['Company Name'].transform('nunique')

ind_d=[]
for i in range(0,counts.shape[0]):
    if counts[i]>1:
        ind_d.append(i)

Space_data.loc[ind_d,['Company Name','Detail','Date']].sort_values('Date').head()

We can see a lot of missions launched the same day from different companies, but the Detail is not similar.

Again, it doesn't seem a good idea to look for the collaborations attending to the Date.

It seams difficult to find the collaborations in this dataset.

# Launches until first success

I'm going to plot the status of the missions

'Success', 'Failure', 'Prelaunch Failure', 'Partial Failure'


In [ ]:
mask_status_s=Space_data['Status Mission']=='Success'
mask_status_f=Space_data['Status Mission']=='Failure'
mask_status_prf=Space_data['Status Mission']=='Prelaunch Failure'
mask_status_pf=Space_data['Status Mission']=='Partial Failure'

plt.figure()
plt.subplots(figsize=(15,14))
plt.plot(Space_data['Date'][mask_status_s],Space_data['Company Name'][mask_status_s],'gd',Space_data['Date'][mask_status_f],Space_data['Company Name'][mask_status_f],'rd',Space_data['Date'][mask_status_prf|mask_status_pf],Space_data['Company Name'][mask_status_prf|mask_status_pf],'yd')
plt.title("Mission Success Based on Date")
plt.xlabel("Year")
plt.ylabel("Company Name")
plt.legend(loc="best");

With this plot we can conclude that the failed missions are more important at the beggining of the Space Race (the earliest dates of the history of space missions), than at the beggining of the individual companies.

There are more mutual information and more experience as time advances and the different companies take proffit of it. This is a general tendence and it's not true for every company, as we can see for ISA.

https://en.wikipedia.org/wiki/Iranian_Space_Agency


In [ ]:

mask_status_ISA=Space_data['Company Name']=='ISA'
Space_data[mask_status_ISA]


I create a DataFrame with the successful missions

In [ ]:
mask_success = Space_data['Status Mission']=='Success'
Space_data_success = Space_data.loc[mask_success]
mask_first = (Space_data_success['Date']).groupby([Space_data_success['Company Name']]).idxmin()
Space_data_first = Space_data_success.loc[mask_first]


I check if it is what I wanted with one example.

In [ ]:
Space_data[Space_data['Company Name']=='ExPace']

In [ ]:
Space_data_first[Space_data['Company Name']=='ExPace']

Having the first successfull mission of each Company, I'm going to look for the previous missions of each Company.

In [ ]:
#mask_not_success = Space_data['Status Mission']!='Success'
#Space_data_not_success = Space_data.loc[mask_not_success]
#Space_data_not_success.head()


When I can't manage with pandas... I make a loop.

In [ ]:
#Space_data_first.head()

In [ ]:
prev=[]

for i in range(0,Space_data.shape[0]):
    Company_ns=Space_data['Company Name'].iloc[i]
    Date_ns=Space_data['Date'].iloc[i]
    Status_ns=Space_data['Status Mission'].iloc[i]

    if Status_ns!='Success':
        one_first=0
        for j in range(0,Space_data_first.shape[0]):
            Company_fs=Space_data_first['Company Name'].iloc[j]
            Status_fs=Space_data_first['Status Mission'].iloc[j]
            Date_fs=Space_data_first['Date'].iloc[j]
            if Company_ns==Company_fs:
                one_first+=1
                if Date_ns<Date_fs:
                    prev.append(i)
                break    
        if one_first==0:
            # it means that this company hasn't no Successful Mission
            prev.append(i)

fail_columns = ['Company Name','Detail','Date','Year','Status Mission']
Space_data_fail_prev=Space_data.loc[prev,fail_columns]#.sort_values('Year').sort_values('Company Name')
Space_data_fail_prev.head()

In [ ]:
#mask_status_s=Space_data['Status Mission']=='Success'
#mask_status_f=Space_data['Status Mission']=='Failure'
#mask_status_prf=Space_data['Status Mission']=='Prelaunch Failure'
#mask_status_pf=Space_data['Status Mission']=='Partial Failure'

plt.figure()
plt.subplots(figsize=(15,10))
plt.plot(Space_data_fail_prev['Date'][mask_status_s],Space_data_fail_prev['Company Name'][mask_status_s],'gd',Space_data_fail_prev['Date'][mask_status_f],Space_data_fail_prev['Company Name'][mask_status_f],'rd',Space_data_fail_prev['Date'][mask_status_prf|mask_status_pf],Space_data_fail_prev['Company Name'][mask_status_prf|mask_status_pf],'yd')
plt.title("Failed missions prior to the first successful mission")
plt.xlabel("Year")
plt.ylabel("Company Name")
plt.legend(loc="best");